## Procesamiento de DataSets


### Procesamiento dataset n1


In [7]:
import csv
from pathlib import Path

path_read= Path ('../datasets/ar-airports.csv')
path_write= Path ('../datasets_custom/ar-airports-custom.csv')


Se hace el import de la libreria csv y de la funcion Path de la liberaria pathlib. Se declara en dos variables diferentes los paths a los archivos.


In [8]:
lectura = open (path_read, mode='r',encoding= 'UTF-8') 
escritura= open(path_write ,mode='w',encoding= 'UTF-8' ,newline= '') 


Se abre el archivo de lectura en modo lectura ('r') y el archivo de escritura en modo escritura ('w'). También se especifica la codificación (UTF-8) y se establece newline=' ' para manejar los saltos de línea.

In [9]:
lectorcsv= csv.reader(lectura)
escritorcsv= csv.writer (escritura)

Se crea un lector CSV (lectorcsv) para leer el archivo de entrada y un escritor CSV (escritorcsv) para escribir en el archivo de salida.

In [10]:
encabezado = next(lectorcsv)
encabezado.append ('elevation_name')
escritorcsv.writerow (encabezado)

260

Se lee la primera fila del archivo de entrada para obtener el encabezado. Luego, se agrega la cadena 'elevation_name' al encabezado. El encabezado actualizado se escribe en el archivo de salida.

In [11]:
for fila in lectorcsv:
    try: 
        elevacion = float(fila[6])
        if elevacion <= 131:
            elevation_name= 'bajo'
        elif elevacion <=903:
            elevation_name= 'medio'
        else:   
            elevation_name='alto'

        fila.append(elevation_name)
        escritorcsv.writerow(fila)

    except ValueError: 
        fila.append ('Sin altura')
        escritorcsv.writerow(fila)

Para cada fila restante en el archivo de entrada se intenta convertir el valor en la sexta columna (la cual es la elevacion del aeropuerto) a un número flotante. Si la conversión es exitosa, se procede a determinar la categoría de elevación ('bajo', 'medio' o 'alto') y se agrega al archivo csv en la columna creada previamente. Si la conversión falla (porque la linea esta vacia o no es un numero), se captura la excepción ValueError y se agrega la cadena 'Sin altura' a la fila.


---

### Procesamiento dataset n2

Haciendo uso de la funcion "create_modified_file()" de modules/Conexiones_internet, se genera un nuevo archivo con una nueva columna de conectividad

In [12]:
from funciones import Conectividad_internet as connectivity
connectivity.create_modified_file()

---

### Procesamiento dataset n3

Primero se importan los módulos y la función con la que se va a trabajar. También se definen las constantes que indican los índices:

In [13]:
import csv
import pathlib
from funciones.lagos_argentina import degrees_to_decimal

SUPERFICIE = 2
COORDENADAS = 5

Utilizando PathLib se crean las rutas del CSV de lectura y del CSV de escritura:

In [14]:
read_dataset = pathlib.Path('../datasets/lagos_arg.csv')
write_dataset = pathlib.Path('../datasets_custom/lagos_arg_custom.csv')

Se abren ambos archivos, uno en modo lectura, otro en modo escritura. Se indica la codificación **UTF-8**. Se usa *newline=''* para evitar líneas vacías entre las filas.

In [15]:
read_file = open(read_dataset, mode='r', encoding='UTF-8', newline='')
write_file = open(write_dataset, mode='w', encoding='UTF-8', newline='')

También se crean los iterables **reader** y **writer** con los que se va a trabajar sobre los archivos: 

In [16]:
reader = csv.reader(read_file)
writer = csv.writer(write_file)

Se toma el *header* del archivo de lectura y se elimina de ésta la columna **COORDENADAS**, ya que va a estar dividida en dos:

In [17]:
reader_header = next(reader)
reader_header.pop (COORDENADAS);

Acto seguido se crea una lista en la que cada elemento es el texto de cada columna nueva. Luego se escribe el *header* del nuevo CSV concatenando la lista del header del archivo original y la lista de los nuevos elementos:

In [18]:
extras_header = ['Sup Tamaño', 'Latitud(GMS)', 'Longitud(GMS)', 'Latitud(GD)', 'Longitud(GD)']
writer.writerow(reader_header + extras_header);

#### Procesamiento

Con un **For** sobre el iterable se realiza el procesamiento del archivo linea por linea.

**-Superficie:** En la variable *sup* se almacena el contenido de la columna *SUPERFICIE* del archivo. Al ser CSV el dato es leído como string, por esto mismo es que se realiza la conversión a entero con *int()*.

Utilizando dicha variable se compara si el valor es chico/medio/grande y se agrega al final de la linea, en la columna de *Sup Tamaño*.


**-Coordenadas:** El String de la columna **COORDENADAS** se divide utilizando *split* (con el espacio entre medio), y ambas partes se guardan en *coordinates*. 

Acto seguido se elimina el String de la columna **COORDENADAS** y se agregan al final los valores separados que están en la lista *coordinates*.

Finalmente empleando la misma lista *coordinates*: Se agregan los valores al final de la línea pero en este caso llamando a la función *degrees_to_decimal*. Esta función toma como argumento el String de coordinadas en formato GMS(grados, minutos y segundos) y lo retorna en GD(grados decimales).

In [19]:
for line in reader:
    # Superficie
    sup = int(line[SUPERFICIE])

    if sup <= 17:
        line.append ('chico')
    elif sup <= 59:
        line.append ('medio')
    else:
        line.append ('grande')

    # Coordenadas
    coordinates = line[COORDENADAS].split(' ')
    line.pop(COORDENADAS)

    line.append (coordinates[0]) 
    line.append (coordinates[1]) 

    line.append (degrees_to_decimal(coordinates[0]))
    line.append (degrees_to_decimal(coordinates[1])) 

    writer.writerow(line)

Para terminar se cierran ambos archivos:

In [20]:
read_file.close()
write_file.close()

---